In [8]:
import pandas as pd
import numpy as np

df_train = pd.read_csv("/content/drive/MyDrive/intern_task_train.csv")
df_test = pd.read_csv("/content/drive/MyDrive/intern_task_test.csv")

In [2]:
from sklearn.metrics import ndcg_score
from sklearn.metrics import accuracy_score


def NDCG_atK_score(test_Data, pred_Data, k, logits=False):
  query_ids = test_Data['query_id'].unique()
  y_true_sessions = [[rank for rank in test_Data[test_Data['query_id'] == id]['rank']] for id in query_ids]
  y_pred_sessions = [[rank for rank in pred_Data[pred_Data['query_id'] == id]['rank']] for id in query_ids]
  if logits:
    print(y_true_sessions)
    print(y_pred_sessions)
  ndcg_scores = [ndcg_score([y_true_sessions[i]], [y_pred_sessions[i]], k=k) for i in range(len(y_true_sessions)) if len(y_true_sessions[i]) > 1]
  return np.mean(ndcg_scores)

In [3]:
!pip install catboost

In [10]:
from catboost import Pool
X_train = df_train.drop(columns=['rank', 'query_id'])
y_train = df_train['rank'] / 5.0
queries_train = df_train['query_id']

X_test = df_test.drop(columns=['rank', 'query_id'])

df_test['rank'] = df_test['rank'] / 5.0

train = Pool(
    data=X_train,
    label=y_train.values,
    group_id=queries_train
)

In [ ]:
from catboost import CatBoostRanker

RESULTS = {}
loss_functions = ['PairLogit', 'PairLogitPairwise', 'YetiRank', 'YetiRankPairwise']
for loss_func in loss_functions:
  model = CatBoostRanker(loss_function=loss_func, task_type="GPU")
  model.fit(train)
  y_pred = model.predict(X_test)
  df_tmp = df_test.copy()
  df_tmp['rank'] = y_pred
  RESULTS[loss_func] = NDCG_atK_score(df_test, df_tmp, k=5)

In [17]:
RESULTS

{'PairLogit': 0.531813173260355,
 'PairLogitPairwise': 0.526202194353459,
 'YetiRank': 0.5313333315941151,
 'YetiRankPairwise': 0.5290864253036566}